### Imports and Setup

In [1]:
import torch
import classiq
from tqdm.auto import tqdm

import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchinfo import summary

from typing import Dict, List
from scripts.helper import create_writer

from classiq import create_model, synthesize, show, QFunc, QArray, QBit, Output, allocate, RX, RY, RZ, RZZ, RXX, RYY, CZ
from classiq.applications.qnn import QLayer
from classiq.execution import execute_qnn
from classiq.synthesis import SerializedQuantumProgram

from classiq.applications.qnn.types import (
    MultipleArguments,
    SavedResult,
    ResultsCollection,
)

/home/devilkillerag/document/python_pro/QNN-MNIST-Classification/qnn/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## Authenticate Classiq
classiq.authenticate()

The current version of 'classiq' has been deprecated, and will not be supported as of 2024-01-18. Please run "pip install -U classiq" to upgrade the classiq SDK to the latest version.
/home/devilkillerag/document/python_pro/QNN-MNIST-Classification/qnn/lib/python3.11/site-packages/classiq/_internals/authentication/token_manager.py:82: UserWarning: Device is already registered.
Generating a new refresh token should only be done if the current refresh token is compromised.
To do so, set the overwrite parameter to true
  warnings.warn(


In [3]:
## For setting up device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device = 'cpu'

In [4]:
## Clear Output Files
post_process_output_file = open("post_process_output.txt", "w")
print("-----------------------------------------------------------------------------------------------------------------", file=post_process_output_file)
print("--------------------------------------------POST PROCESS OUTPUT--------------------------------------------------", file=post_process_output_file)
print("-----------------------------------------------------------------------------------------------------------------", file=post_process_output_file)
post_process_output_file.close()

test_loop_output_file = open("test_loop_output.txt", "w")
print("-----------------------------------------------------------------------------------------------------------------", file=test_loop_output_file)
print("-----------------------------------------------TEST LOOP OUTPUT--------------------------------------------------", file=test_loop_output_file)
print("-----------------------------------------------------------------------------------------------------------------", file=test_loop_output_file)
test_loop_output_file.close()

### Preparing Data

#### Getting a Dataset

In [5]:
def input_transform(image):
    """
    The input MNIST images are all 28 × 28 px. This function will firstly center-crop 
    them to 24 × 24 and then down-sample them to 4 × 4 for MNIST. Then we convert 
    the image pixels into angles for passing them into Rotation gates later for encoding.
    """
    image = transforms.ToTensor()(image)
    image = transforms.CenterCrop(24)(image)
    image = transforms.Resize(size = (4,4))(image)
    image = image.squeeze()
    image_pixels = torch.flatten(image)
    angles = torch.sqrt(image_pixels / 256)
    
    return angles

In [6]:
def target_transform(label):
    label_tensor = torch.LongTensor([label])
    one_hot_label = torch.nn.functional.one_hot(label_tensor, 10)
    return one_hot_label.squeeze()

In [7]:
# Setup training data
train_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=input_transform,
    target_transform=target_transform
)

# Setup testing data
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=input_transform,
    target_transform=target_transform
)

#### Visualize the Data

In [8]:
len(train_data), len(test_data)

(60000, 10000)

In [9]:
# See the first training example
image, label = train_data[0]
image, label

/home/devilkillerag/document/python_pro/QNN-MNIST-Classification/qnn/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


(tensor([0.0000, 0.0000, 0.0317, 0.0378, 0.0000, 0.0336, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0477, 0.0000, 0.0295, 0.0620, 0.0000, 0.0000]),
 tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0]))

In [10]:
# See a random training example
import random
image, label = train_data[random.randint(0, len(train_data))]
image, label

(tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0167, 0.0338, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0401, 0.0000, 0.0000]),
 tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0]))

#### Prepare Dataloader

In [11]:
from torch.utils.data import Subset

# Define the size of the subset
subset_size = 64

# Create subsets of the datasets
train_subset = Subset(train_data, range(subset_size))
test_subset = Subset(test_data, range(subset_size))

In [12]:
# Setup the batch size hyperparameter
BATCH_SIZE = 32

# Turn datasets into iterables (batches)
train_dataloader = DataLoader(train_subset,
    batch_size=BATCH_SIZE, 
    shuffle=True
)

test_dataloader = DataLoader(test_subset,
    batch_size=BATCH_SIZE,
    shuffle=False
)

In [13]:
# Let's check out what we've created
print(f"Dataloaders: {train_dataloader, test_dataloader}") 
print(f"Length of train dataloader: {len(train_dataloader)} batches of {BATCH_SIZE}")
print(f"Length of test dataloader: {len(test_dataloader)} batches of {BATCH_SIZE}")

Dataloaders: (<torch.utils.data.dataloader.DataLoader object at 0x7f9c13226950>, <torch.utils.data.dataloader.DataLoader object at 0x7f9c131f5590>)
Length of train dataloader: 2 batches of 32
Length of test dataloader: 2 batches of 32


In [14]:
data, label = next(iter(train_dataloader))

print(f"Image shape: {data.shape} -> [batch_size, pixel_angle]")
print(f"Label shape: {label.shape} -> [batch_size, label_value]")

Image shape: torch.Size([32, 16]) -> [batch_size, pixel_angle]
Label shape: torch.Size([32, 10]) -> [batch_size, label_value]


### Quantum Model

In [15]:
@QFunc
def encoding(q: QArray[QBit]) -> None:
    """
    This function encodes the input data into the qubits. This input data is a 4x4 image pixel values 
    converted into angle for rotation gates (RX, RY, RZ, RX) in form of a 16x1 vector. 
    We encode 4 pixels per qubit.

    Args:
        q (QArray[QBit]): Array of four Qubits to encode the input data into.
    """
    RX(theta="input_0", target=q[0]) # Pixel 0 on Qubit 0
    RY(theta="input_1", target=q[0]) # Pixel 1 on Qubit 0
    RZ(theta="input_2", target=q[0]) # Pixel 2 on Qubit 0
    RX(theta="input_3", target=q[0]) # Pixel 3 on Qubit 0
    
    RX(theta="input_4", target=q[1]) # Pixel 4 on Qubit 1
    RY(theta="input_5", target=q[1]) # Pixel 5 on Qubit 1
    RZ(theta="input_6", target=q[1]) # Pixel 6 on Qubit 1
    RX(theta="input_7", target=q[1]) # Pixel 7 on Qubit 1
    
    RX(theta="input_8", target=q[2]) # Pixel 8 on Qubit 2
    RY(theta="input_9", target=q[2]) # Pixel 9 on Qubit 2
    RZ(theta="input_10", target=q[2]) # Pixel 10 on Qubit 2
    RX(theta="input_11", target=q[2]) # Pixel 11 on Qubit 2
    
    RX(theta="input_12", target=q[3]) # Pixel 12 on Qubit 3
    RY(theta="input_13", target=q[3]) # Pixel 13 on Qubit 3
    RZ(theta="input_14", target=q[3]) # Pixel 14 on Qubit 3
    RX(theta="input_15", target=q[3]) # Pixel 15 on Qubit 3


In [16]:
@QFunc
def mixing(q: QArray[QBit]) -> None:
    """
    This function performs the mixing operation on the qubits. 
    This is done by applying a series of RZZ, RXX, RYY gates to form a
    ring connection.

    Args:
        q (QArray[QBit]): Array of four Qubits to apply the mixing operation on.
    """
    RZZ(theta="weight_0", target=q[0:2])
    RZZ(theta="weight_1", target=q[1:3])
    RZZ(theta="weight_2", target=q[2:4])
    
    RXX(theta="weight_4", target=q[0:2])
    RXX(theta="weight_5", target=q[1:3])
    RXX(theta="weight_6", target=q[2:4])
    
    RYY(theta="weight_8", target=q[0:2])
    RYY(theta="weight_9", target=q[1:3])
    RYY(theta="weight_10", target=q[2:4])

In [17]:
@QFunc
def cz_block(q: QArray[QBit]) -> None:
    """
    This function applies CZ gates between each qubit.

    Args:
        q (QArray[QBit]): Array of four Qubits to apply the entanglement operation on.
    """
    CZ(control=q[0], target=q[1])
    CZ(control=q[1], target=q[2])
    CZ(control=q[2], target=q[3])

In [18]:
@QFunc
def main(res: Output[QArray[QBit]]) -> None:
    """
    This is the main function from which model will be created. 
    It calls the other functions to perform the encoding, mixing and entanglement.

    Args:
        res (Output[QArray[QBit]]): Output QArray of QBits from which the model will be created.
    """
    allocate(4, res)
    encoding(q=res)
    mixing(q=res)
    cz_block(q=res)

In [19]:
# Create a model
model = create_model(main)
quantum_program = synthesize(model)
show(quantum_program)

Opening: https://platform.classiq.io/circuit/59626f2b-bebb-4a9c-ac56-af30e550cb58?version=0.33.0


### Quantum Neural Network

In [20]:
def execute(quantum_program: SerializedQuantumProgram, arguments: MultipleArguments) -> ResultsCollection:
    return execute_qnn(quantum_program, arguments)

In [21]:
def post_process(result: SavedResult) -> torch.Tensor:
    counts: dict = result.value.counts
    
    # Calculate logits from counts
    logits: float = torch.zeros(16)
    for key, value in counts.items():
        logits[int(key, 2)] = value
    
    # Trim the logits from length 16 to length 10 since we have only 10 labels
    trimmed_logits = logits[:10]
    
    # Calculate prediction probabilities from logits by normalizing it
    pred_probs = torch.nn.functional.normalize(trimmed_logits, dim=0)
    
    # Convert the prediction probabilities into prediction labels
    pred_labels = torch.argmax(pred_probs)
    
    ### WRITE COUNTS, OUTPUT LOGITS, PRED PROBS, PRED LABELS to a file
    output_file = open("post_process_output.txt", "a")
    print("----------------------------------------------------------------------------------------------------------------------------------------------", file=output_file)
    print(f"COUNTS:: \n {counts} \n", file=output_file)
    print(f"LOGITS:: \n {logits} \n", file=output_file)
    print(f"TRIMMED LOGITS:: \n {trimmed_logits} \n", file=output_file)
    print(f"PREDICTION PROBABILITIES:: \n {pred_probs} \n", file=output_file)
    print(f"PREDICTION LABELS:: \n {pred_labels} \n", file=output_file)
    output_file.close()
    
    return torch.tensor(pred_probs)

In [22]:
class Net(torch.nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__()
        self.qlayer = QLayer(
            quantum_program,
            execute,
            post_process,
            *args,
            **kwargs
        )
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.qlayer(x)
        return x

gio: https://platform.classiq.io/circuit/59626f2b-bebb-4a9c-ac56-af30e550cb58?version=0.33.0: Operation not supported


In [23]:
qnn = Net()

In [24]:
# summary(model=qnn, input_size=(32, 16), verbose=0, col_names=["input_size", "output_size", "num_params", "trainable"], col_width=20, row_settings=["var_names"])

### Training and Testing Loop

In [25]:
_LEARNING_RATE = 1.0

# choosing our loss function
loss_fn = nn.L1Loss()
# choosing our optimizer
optimizer = optim.SGD(qnn.parameters(), lr=_LEARNING_RATE)

In [26]:
def train(
    model: nn.Module,
    data_loader: DataLoader,
    loss_fn: nn.modules.loss._Loss,
    optimizer: optim.Optimizer,
    writer: torch.utils.tensorboard.writer.SummaryWriter,
    epochs: int = 20,
) -> Dict[str, List]:
    model.to(device)
    
    # Setup train loss value
    train_loss = 0
    
    # Create empty results dictionary
    results = {
        "train_loss": [],
    }
    
    # Loop through training steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        print(f"Epoch: {epoch}\n----------")
        for batch, (data, label) in enumerate(data_loader):
            # Send data to device (GPU or CPU)
            data, label = data.to(device), label.to(device)
            
            # 1. Forward pass
            output = model(data)
            
            # 2. Calculate loss
            loss = loss_fn(output, label)
            train_loss += loss
            
            # 3. Optimizer zero grad
            optimizer.zero_grad()
            
            # 4. Loss backward
            loss.backward()
            
            # 5. Optimizer step
            optimizer.step()
        
        # Calculate loss per epoch and print out what's happening
        train_loss /= len(data_loader)
        
        # Print out what's happening
        print(
            f"Epoch: {epoch+1} | "
            f"Train loss: {train_loss:.5f}"
        )
        
        # Update results dictionary
        results["train_loss"].append(train_loss)
        
        ### Experiment Tracking ###
        # See if there's a writer, if so, log to it
        if writer:
            # Add loss results to SummaryWriter
            writer.add_scalars(
                main_tag="Loss", 
                tag_scalar_dict={"train_loss": train_loss,},
                global_step=epoch
            )
            
            # Close the writer
            writer.close()
    
    # Return the filled results at the end of the epochs
    return results

In [27]:
def test(
    model: nn.Module,
    data_loader: DataLoader,
    atol=0
) -> float:
    num_correct = 0
    total = 0
    
    # Put the model in eval mode
    model.eval()
    
    # Turn on inference mode context manager
    with torch.inference_mode():
        for data, labels in data_loader:
            # Send data to GPU
            data, labels = data.to(device), labels.to(device)
            
            # 1. Forward pass: Let the model predict
            predictions = model(data)
            
            # Get a tensor of booleans, indicating if each label is close to the real label
            is_prediction_correct = torch.isclose(predictions.argmax(dim=1), labels.argmax(dim=1), atol=atol)
            output_file = open("test_loop_output.txt", "a")
            print("----------------------------------------------------------------------------------------------------------------------------------------------", file=output_file)
            print(f"LABELS:: \n {labels} \n", file=output_file)
            print(f"PREDICTIONS:: \n {predictions} \n", file=output_file)
            print(f"IS PREDICTIONS CORRECT:: \n {is_prediction_correct} \n", file=output_file)
            output_file.close()
            
            # Count the amount of `True` predictions
            num_correct += is_prediction_correct.sum().item()
            
            # Count the total evaluations
            #   the first dimension of `labels` is `batch_size`
            total += labels.size(0)
    
    # Calculate the accuracy
    accuracy = float(num_correct) / float(total)
    print(f"Test Accuracy of the model: {accuracy * 100:.2f}%")
    return accuracy * 100

#### Run Experiment

##### 01. Train

In [28]:
# Create a writer for tracking our experiment
writer = create_writer(experiment_name="data_0.1_percent", model_name="linear_entanglement_x3", extra="2_epochs")

[INFO] Created SummaryWriter, saving to: runs/2024-01-04/data_0.1_percent/linear_entanglement_x3/2_epochs...


In [29]:
results = train(qnn, train_dataloader, loss_fn, optimizer, writer=writer, epochs=2)

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 0
----------


/home/devilkillerag/document/python_pro/QNN-MNIST-Classification/qnn/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/tmp/ipykernel_63607/4157040924.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceT

Epoch: 1 | Train loss: 0.19132
Epoch: 1
----------


100%|██████████| 2/2 [02:24<00:00, 72.30s/it]

Epoch: 2 | Train loss: 0.28401


In [30]:
# Check out the model results
results

{'train_loss': [tensor(0.2840, grad_fn=<DivBackward0>),
  tensor(0.2840, grad_fn=<DivBackward0>)]}

In [31]:
# %load_ext tensorboard
# %tensorboard --logdir runs

##### 02. Test

In [32]:
test(qnn, test_dataloader)

/tmp/ipykernel_63607/4157040924.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(pred_probs)


Test Accuracy of the model: 9.38%


9.375

: 